In [4]:
import json, sys, copy, os, igl
import numpy as np
import meshio

def is_inside(v, f, p):
    winds = igl.winding_number(v, f, p)
    mask = np.array(np.rint(winds), dtype=int)
    
    return np.array([e % 2 == 1 for e in mask])

def change_file_extension(filename, new_extension):
  base, ext = os.path.splitext(filename)
  return base + "." + new_extension if ext else filename + "." + new_extension

In [5]:
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
microstructure_repo_path = notebook_dir
matopt_repo_path = os.path.join(os.path.dirname(microstructure_repo_path), "matopt")

input_surface = "/home/luce/code/thimble/cad/thimble-no-magnets.stl"  # USER: Change this to your STL file path
stitch_cells_cli = os.path.join(microstructure_repo_path, "build/isosurface_inflator/cut_cells_cli")
only_cube_cells = False

In [ ]:
E = 0.01
nu = 0.09
cell_size = 8
# out_path = str(E) + "_" + str(cell_size) + ".obj"
out_path = f"{input_surface[:-4]}_{E}_{cell_size}.obj_{only_cube_cells}.obj"
print(out_path)

m = meshio.read(input_surface)
input_surface = change_file_extension(input_surface, 'obj')
m.write(input_surface)

# get mesh vertices and faces
v = m.points.astype(float)
f = m.cells[0].data

# compute bounding box
bbox = [np.amin(v, axis=0), np.amax(v, axis=0)]

# discretise box into grids of cells
corner0 = list(map(int, np.ceil(bbox[0] / cell_size)))
# corner0 = list(map(int, np.ceil(bbox[0] / cell_size) - 1))
corner1 = list(map(int, np.floor(bbox[1] / cell_size)))

print(corner0, corner1)

# use return E for uniform Young's modulus
# use i,j,k to vary the Young's modulus through the dimensions
def young(i, j, k):
    # return E
    print(f"young i, j, k: [{i, j, k}]")
    if k == 0:
        return 0.001
    if k == 1:
        return 0.0015
    if k == 2:
        return 0.002
    if k == 3:  
        return 0.005
    if k == 4:
        return 0.005
    if k == 5:
        return 0.005
    if k == 6:
        return 0.005
    if k == 7:
        return 0.005
    if k == 8:
        return 0.005
    if k == 9:
        return 0.005
    
    

/home/luce/code/thimble/cad/thimble-no-magnets_0.01_8.obj_False.obj
[-4, -4, -1] [3, 3, 3]


In [ ]:
sys.path.insert(0, os.path.join(matopt_repo_path, 'tools/material2geometry'))
from material2geometry import Material2Geometry

In [ ]:
mat2geo = Material2Geometry(in_path=os.path.join(matopt_repo_path, "tools/material2geometry/0646_geo_1_coeffs.txt"))

In [ ]:
patterns = []
entry = {"params": [],
"symmetry": "Cubic",
"pattern": os.path.join(microstructure_repo_path, "data/patterns/3D/reference_wires/pattern0646.wire"),
"index": [0,0,0]}

for i in range(corner0[0], 1 + corner1[0]):
    for j in range(corner0[1], 1 + corner1[1]):
        for k in range(corner0[2], 1 + corner1[2]):
            geo_params = mat2geo.evaluate(nu, young(i, j, k))
            entry["params"] = geo_params
            entry["index"] = [i,j,k]
            patterns.append(copy.deepcopy(entry))

with open("data.json", 'w') as f:
    json.dump(patterns, f)

In [ ]:
os.system(stitch_cells_cli + " -p data.json " +
           (("--surface " + input_surface) if not only_cube_cells else "") + 
           " --gridSize " + str(cell_size) + " -o " + out_path + " -r 50")
# print((stitch_cells_cli + " -p data.json " +
#            (("--surface " + input_surface) if not only_cube_cells else "") + 
#            " --gridSize " + str(cell_size) + " -o " + out_path + " -r 50"))

In [ ]:
for c in [1, 2, 3, 4, 5, 6, 7, 8]:
    for E in [1e-3, 2e-3, 5e-3, 1e-2]:
        print(f"Processing for c={c} and E={E}")
        print((np.array(mat2geo.evaluate(nu, E)[5:-4]) * c).tolist())

In [ ]:
# To check printability of the geometry, we can create a DataFrame to summarize the minimum dimensions for different cell sizes and Young's moduli.Based on your printer nozzle size, we can determine if the geometry is printable or not.

import pandas as pd
import numpy as np

c_values = [1, 2, 3, 4, 5, 6, 7, 8, 16]
E_values = [1e-3, 2e-3, 5e-3, 1e-2, 2e-2]

df = pd.DataFrame(index=c_values, columns=E_values)

df.index.name = "Cell Size (c)"
df.columns.name = "Young's Modulus (E)"

for c in c_values:
    for E in E_values:
        arr = np.array(mat2geo.evaluate(nu, E)[5:-4]) * c
        min_dim = np.min(arr)        
        if min_dim < 0.4:
            df.loc[c, E] = "Unprintable"
        else:
            df.loc[c, E] = f"{min_dim:.4f}"

df